<h1>1) Review the movie names and languages names in the year of 2019.</h1>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, when, countDistinct, collect_list
spark = SparkSession.builder.appName("AmazonAnalysis").getOrCreate()
amazon_prime_movies_df = spark.read.csv("amazon_cleaned_data.csv", header=True, inferSchema=True)

movies_2019_df = amazon_prime_movies_df.filter(col("year_of_release") == '2019').select("movie_name", "language")

movies_2019_df.show()
movies_2019_df.count()


+--------------------+--------+
|          movie_name|language|
+--------------------+--------+
|           Bombshell| English|
|               Hippi|  Telugu|
|         Nanna Prema|  Telugu|
|    City of football| Bengali|
|Eye To Eye With E...| English|
|Kanhaiya Ek Yodha...|   Hindi|
|            Yajamana| Kannada|
|              Kharij| Bengali|
|Dash Mash Dash Di...| Bengali|
|Bodhai Yeri Budhi...|   Tamil|
|Manikarnika: The ...|   Tamil|
|     The Cabin Guard| Bengali|
|Sharkwater Extinc...| English|
|Zombieland: Doubl...|  Telugu|
|Invisible Wildlif...| English|
|Letters From Gene...| English|
|Captain Monsteric...| English|
|            Doorbeen| Punjabi|
|YASHKA, BATTALION...| English|
|Mongolia - In The...| English|
+--------------------+--------+
only showing top 20 rows


691

<h1>2) Which are the languages, movies highly watched by people of all ages? And in this scenrio, give the movie names of that language.</h1>

In [3]:
from pyspark.sql.functions import trim
# Filter movies with 'All' maturity rating
all_ages_df = amazon_prime_movies_df.filter(trim(col("maturity_rating")) == "All")

# Count movies by language
language_counts = all_ages_df.groupBy("language").count()

# Find the language with the highest count
top_language = language_counts.orderBy(col("count").desc()).first()["language"]

# Get movie names in that language with 'All' rating
top_language_movies = all_ages_df.filter(col("language") == top_language).select("movie_name")

# Show results
print(f"Most common language for 'All' rated movies: {top_language}")
top_language_movies.show(truncate=False)


Most common language for 'All' rated movies: Hindi
+-------------------------------+
|movie_name                     |
+-------------------------------+
|Tum Se Achcha Kaun Hai         |
|Ekkees Tareekh Shubh Muhurat   |
|Devdas                         |
|Ramgarh Ke Sholay              |
|Mere Yaar Ki Shaadi Hai        |
|Swarg                          |
|Jeevan Ek Sanghursh            |
|Khuli Khidki - Pratibha Sharma |
|Love 86                        |
|Jagriti                        |
|Chhota Bheem - Romani Adventure|
|Ek Phool Do Mali               |
|Agnipankh                      |
|The Peanut Seller              |
|Chitchor                       |
|Kanoon                         |
|Srinivasa Kalyanam             |
|Bobby Jasoos                   |
|Prem Ratan Dhan Payo           |
|Kuch Tum Kaho Kuch Hum Kahein  |
+-------------------------------+
only showing top 20 rows


<h1>3) Which age group (maturity rating) is watching hindi movies? also, the movie titles.</h1>

In [4]:

# Filter for Hindi movies
hindi_movies = amazon_prime_movies_df.filter(col("language") == "Hindi")

# Select distinct age groups and movie titles
result = hindi_movies.select("maturity_rating", "movie_name").distinct()

# Show the result
result.show()



+---------------+--------------------+
|maturity_rating|          movie_name|
+---------------+--------------------+
|            All|            Xcuse Me|
|            13+|                 Dus|
|            13+|              Pinjar|
|             NR|   Ek Din Aadhi Raat|
|             NR|            Bezubaan|
|            All|            Notebook|
|            All|       Green Lantern|
|            All|              Kanoon|
|            All|               Alaap|
|            All|Chhota Bheem Vs Z...|
|            All|Chhota Bheem Aur ...|
|             NR|That Man In The P...|
|            18+|       Chudail Story|
|            All|           Parasmani|
|            18+|               Aatma|
|             NR|             Sabrang|
|             7+|     Despicable Me 2|
|            18+| No Strings Attached|
|             NR|              Sanjog|
|             NR|          Nai Roshni|
+---------------+--------------------+
only showing top 20 rows


<h1>4) Which movie has the highest running duration and in which year?</h1>

In [5]:
from pyspark.sql.functions import max, col

max_duration = amazon_prime_movies_df.agg(max("total_running_time_minutes").alias("max_duration")).collect()[0]["max_duration"]

# Filter the movie(s) with the maximum duration
longest_movie_df = amazon_prime_movies_df.filter(col("total_running_time_minutes") == max_duration)

# Show the result
longest_movie_df.select("movie_name", "year_of_release", "total_running_time_minutes").show()



+--------------------+---------------+--------------------------+
|          movie_name|year_of_release|total_running_time_minutes|
+--------------------+---------------+--------------------------+
|Once Upon a Time ...|           1984|                       229|
+--------------------+---------------+--------------------------+



<h1>5) And review the total number of movie ratings by language wise, and what are the movies that are high maturity rating?</h1>

In [6]:

from pyspark.sql.functions import countDistinct, desc, count
# Total number of movie ratings by language
ratings_by_language = amazon_prime_movies_df.groupBy("language").agg(count("maturity_rating").alias("total_ratings"))
ratings_by_language.show()

# Movies with high maturity rating (assuming "R" and "NC-17" are high maturity)
high_maturity_movies = amazon_prime_movies_df.filter(col("maturity_rating").isin("All", "18+")).select("movie_name", "maturity_rating")
high_maturity_movies.show()



+---------+-------------+
| language|total_ratings|
+---------+-------------+
|  Kannada|          394|
| Gujarati|          104|
|   Telugu|          928|
|  English|         1583|
|  Marathi|          317|
|    Tamil|          661|
|  Bengali|         1124|
|    Hindi|         1827|
|  Punjabi|          121|
|Malayalam|          482|
+---------+-------------+

+--------------------+---------------+
|          movie_name|maturity_rating|
+--------------------+---------------+
|          John Rambo|            18+|
|        American Pie|            18+|
|           Bombshell|            18+|
|          Love Birds|            All|
|               Hippi|            18+|
|               Ayyaa|            All|
|Tum Se Achcha Kau...|            All|
|               Swamy|            All|
|Ekkees Tareekh Sh...|            All|
|       Ghost Stories|            18+|
|              Devdas|            All|
|        Akashadoothu|            All|
|  Mera Pind: My Home|            All|
|300: Rise 